In [ ]:
!pip install -q torch transformers accelerate bitsandbytes sentence-transformers faiss-gpu-cu11

1 - Data preprocessing\
2 - Integrating RAG \
3 - Plagiarism agent \
4 - using api


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain
!pip install -U langchain-community langchain-openai langchainhub -q
!pip install gradio --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from langchain.llms import OpenAI as LangchainOpenAI
from langchain_openai import ChatOpenAI
from getpass import getpass

# Get API key
api_key = getpass('Enter your OpenAI API Key: ')

# Create LangChain OpenAI instance with the API key
llm= ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, temperature=0.1)
# llm = LangchainOpenAI(model="gpt-4o-mini",openai_api_key=api_key, temperature=0.1)

Enter your OpenAI API Key: ··········


In [ ]:
# Prompt template for our model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
# 1. summary chain
summary_prompt = PromptTemplate(
    input_variables=["essay"],
    template="Summarize the following essay in 2-3 sentences:\n\n{essay}"
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")


<ipython-input-5-77f03f793e7e>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")


In [ ]:
# 2. rubric chain
scoring_prompt = PromptTemplate(
    input_variables=["essay"],
    template="""Evaluate the following essay on a scale of 0-10 for each of these five categories:

    1. Thesis & Main Argument
    2. Evidence & Support
    3. Organization & Structure
    4. Critical Thinking
    5. Writing Mechanics

    Provide a score and a brief justification for each:\n\n{essay}"""
)
scoring_chain = LLMChain(llm=llm, prompt=scoring_prompt, output_key="scores")


In [ ]:
# 3. overall score chain
overall_score_prompt = PromptTemplate(
    input_variables=["scores"],
    template="Based on the following rubric scores, calculate an overall score out of 50:\n\n{scores}, write the title and the score without detal, make it easy to read by the users"
)
overall_score_chain = LLMChain(llm=llm, prompt=overall_score_prompt, output_key="overall_score")

In [ ]:
# 4. strengths chain
strengths_prompt = PromptTemplate(
    input_variables=["essay"],
    template="List three specific strengths of the following essay:\n\n{essay}"
)
strengths_chain = LLMChain(llm=llm, prompt=strengths_prompt, output_key="strengths")


In [ ]:
# 5. improvement chain
improvements_prompt = PromptTemplate(
    input_variables=["essay"],
    template="""List three actionable areas for improvement in the following essay, along with concrete
    examples of how the writer can improve:\n\n{essay}"""
)
improvements_chain = LLMChain(llm=llm, prompt=improvements_prompt, output_key="improvements")


In [ ]:
# 6.  final feedback chain
feedback_prompt = PromptTemplate(
    input_variables=["summary", "scores", "strengths", "improvements"],
    template="""Based on the summary, rubric scores, strengths, and areas for improvement, provide a final
    paragraph of constructive feedback focusing on the most impactful ways the writer can improve:\n
    Summary: {summary}
    Scores: {scores}
    Strengths: {strengths}
    Areas for Improvement: {improvements}"""
)
feedback_chain = LLMChain(llm=llm, prompt=feedback_prompt, output_key="feedback")

In [ ]:
grading_chain = SequentialChain(
    chains=[summary_chain, scoring_chain, overall_score_chain, strengths_chain, improvements_chain, feedback_chain],
    input_variables=["essay"],
    output_variables=["summary", "scores", "overall_score", "strengths", "improvements", "feedback"]
)

In [ ]:
essay_text = input("Your essay here...")
# For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

# I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

# I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts because I like framing my life in this way. However, this is a personal choice - you don’t have to do the same.

# For me, the most formative experiences included, in no specific order:

# Participating in five All-Russian Olympiads in law (three in high school, two at Moscow State University), winning two first prizes and three third prizes. The one I won at 17 was especially impactful because it allowed me to choose any law faculty in the country without exams.
# Competing in moot courts, specifically the Concours Charles-Rousseau and the Philip C. Jessup competitions. I also coached a team for the Charles-Rousseau for three years.
# Growing up in extreme poverty with my father
# Teaching law of obligations to third-year students at Moscow State University.
# Working as a researcher at the Higher School of Economics, contributing to projects for the Russian Central Bank and participating in the banking law reform.
# Learning German and French through three university classes and self-study to participate in academic programs abroad.
# Working at a German law firm for three years.
# I will elaborate on a few of these experiences. You'll notice my descriptions are fairly detailed - not to overindulge myself to show just how much context I had to explore and analyze before I choosing stories for my personal statement. I recommend doing the same for your essays. This approach can help you feel more confident about your profile and make your choices more strategic and informed."""
result = grading_chain(essay_text)
result.pop("essay", None)
print(result)

Your essay here...Japan’s cherry blossoms, known as sakura, are one of the country’s most iconic symbols. These delicate pink and white flowers bloom in early spring, transforming parks and streets into breathtaking landscapes. Beyond their beauty, sakura hold deep cultural and historical significance in Japan, representing themes of renewal, impermanence, and unity.  The fleeting nature of cherry blossoms, which bloom for only a short period before falling, reflects the Japanese concept of mono no aware—an appreciation for the transient beauty of life. This idea is deeply ingrained in Japanese culture, reminding people to cherish the present moment.  Hanami, or "flower viewing," is a beloved tradition in Japan where families and friends gather under the cherry trees for picnics and celebrations. This practice dates back to the Heian period (794–1185), when aristocrats admired the blossoms while composing poetry. Today, hanami is a nationwide event, with major parks in Tokyo, Kyoto, an

<ipython-input-16-55473c4287b7>:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = grading_chain(essay_text)


{'summary': "Japan's cherry blossoms, or sakura, are a symbol of beauty and cultural significance, representing themes of renewal and impermanence. Their brief blooming period reflects the Japanese concept of mono no aware, emphasizing the appreciation of life's transient moments. The tradition of hanami, where people gather to enjoy the blossoms, highlights their importance in Japanese culture, art, and even commerce, making sakura a profound representation of the nation's values and spirit.", 'scores': 'Here’s an evaluation of the essay based on the specified categories:\n\n1. **Thesis & Main Argument: 8/10**\n   - **Justification:** The thesis is clear and presents a strong main argument about the cultural and historical significance of cherry blossoms in Japan. It effectively sets the stage for the discussion that follows. However, it could be slightly more explicit in outlining the specific points that will be addressed in the essay.\n\n2. **Evidence & Support: 7/10**\n   - **Just

In [ ]:
import gradio as gr
from langchain.chains import SequentialChain

# Function to process essay
def process_essay(essay):
    try:
        results = grading_chain.invoke({"essay": essay})

        # Format the output properly to avoid unintended code blocks
        formatted_output = (
            "**Summary**\n\n"
            f"{results['summary'].strip()}\n\n"
            "**Detailed Scores**\n\n"
            f"{results['scores'].strip()}\n\n"
            "**Overall Score**\n\n"
            f"{results['overall_score'].strip()}\n\n"
            "**Key Strengths**\n\n"
            f"{results['strengths'].strip()}\n\n"
            "**Areas for Improvement**\n\n"
            f"{results['improvements'].strip()}\n\n"
            "**Overall Feedback**\n\n"
            f"{results['feedback'].strip()}"
        )

        return formatted_output

    except Exception as e:
        return f"An error occurred: {str(e)}"


# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Essay Grading System")
    essay_input = gr.Textbox(label="Enter your essay here")
    output_display = gr.Markdown()  # Use Markdown instead of Textbox for proper formatting
    submit_button = gr.Button("Submit")

    submit_button.click(process_essay, inputs=essay_input, outputs=output_display)

if __name__ == "__main__":
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c716a7eac57aa81d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
 t

In [ ]:


# prompt = PromptTemplate(
#     input_variables=['essay_text'],
#     template='''You are an experienced essay grader. When given an essay, analyze it according to the following rubric, scoring each criterion from 0-10:

# 1. Thesis & Main Argument
# - Clear, specific thesis statement
# - Consistent focus throughout the essay
# - Well-developed main arguments

# 2. Evidence & Support
# - Relevant examples and evidence
# - Proper citation of sources
# - Effective integration of quotes/evidence

# 3. Organization & Structure
# - Logical flow between paragraphs
# - Strong topic sentences
# - Effective transitions
# - Clear introduction and conclusion

# 4. Critical Thinking
# - Depth of analysis
# - Consideration of counterarguments
# - Original insights
# - Logical reasoning

# 5. Writing Mechanics
# - Grammar and spelling
# - Sentence structure variety
# - Academic vocabulary
# - Proper formatting

# For each essay, provide:

# 1. A brief summary (2-3 sentences) of the main arguments and key points
# 2. Scores for each rubric criterion (0-10) with brief justification
# 3. An overall score out of 50
# 4. Three specific strengths of the essay
# 5. Three actionable areas for improvement, with concrete examples of how to implement each suggestion
# 6. A final paragraph of constructive feedback focusing on the most impactful ways the writer can improve their next essay

# Maintain a supportive but professional tone. Focus on specific, implementable feedback rather than general comments. When identifying areas for improvement, always provide examples of how to address them.

# Here are examples of essays and how they should be evaluated:

# GOOD ESSAY EXAMPLE:

# [Walking into the voting booth "...every four years on the Tuesday after the first Monday in November" (Office Of Federal Register 6), the people of the United States of America expect their vote to count. Although, the president is elected in a more indirect way, by the electoral college, which does not represent the people, and is not fair. The president should be elected by popular vote from now on...]

# This essay demonstrates strong argumentation, clear structure, and effective use of evidence. Note the clear thesis statement, logical organization, and proper citation of sources. The analysis is thorough and considers multiple aspects of the electoral college system.

# POOR ESSAY EXAMPLE:

# [When sitting in class do you ever feel like you can read someones emotion, well now you can. With the new program Facial Action Coding System you can make a 3-D model of someone and scan the face mucsles to tell you what emotions they are feeling...]

# This essay shows weaknesses in organization, lacks proper evidence, and has multiple mechanical errors. Note the informal tone, lack of clear thesis, and absence of proper paragraph structure and transitions.

# Please grade the following essay:

# {essay_text}''',
# )

# #prompt.format(essay_text = "Your essay text here")


In [ ]:
# from langchain.chains import LLMChain
# chain = LLMChain(llm=llm, prompt=prompt)
# essay = input("Please insert your essay here!")
# # chain.run(essay)

In [ ]:
# llm = HuggingFaceHub(
#             repo_id= "meta-llama/Llama-3.1-8B-chat-hf",  # e.g., "meta-llama/Llama-2-70b-chat-hf"
#             model_kwargs={
#                 "temperature": 0.1,  # Low temperature for more consistent grading
#                 "max_new_tokens": 2000,
#                 "top_p": 0.95
#             }
#         )

In [ ]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(
    repo="huggingface/peft",
    access_token=ACCESS_TOKEN,
    include_prs=False,
    state="all"
)

docs = loader.load()

NameError: name 'ACCESS_TOKEN' is not defined

In [ ]:
essay_grader_prompt = ("""
You are an experienced essay grader. When given an essay, analyze it according to the following rubric, scoring each criterion from 0-10:

1. Thesis & Main Argument
- Clear, specific thesis statement
- Consistent focus throughout the essay
- Well-developed main arguments

2. Evidence & Support
- Relevant examples and evidence
- Proper citation of sources
- Effective integration of quotes/evidence

3. Organization & Structure
- Logical flow between paragraphs
- Strong topic sentences
- Effective transitions
- Clear introduction and conclusion

4. Critical Thinking
- Depth of analysis
- Consideration of counterarguments
- Original insights
- Logical reasoning

5. Writing Mechanics
- Grammar and spelling
- Sentence structure variety
- Academic vocabulary
- Proper formatting

For each essay, provide:

1. A brief summary (2-3 sentences) of the main arguments and key points
2. Scores for each rubric criterion (0-10) with brief justification
3. An overall score out of 50
4. Three specific strengths of the essay
5. Three actionable areas for improvement, with concrete examples of how to implement each suggestion
6. A final paragraph of constructive feedback focusing on the most impactful ways the writer can improve their next essay

Maintain a supportive but professional tone. Focus on specific, implementable feedback rather than general comments. When identifying areas for improvement, always provide examples of how to address them.

Here are examples of essays and how they should be evaluated:

GOOD ESSAY EXAMPLE:

[Walking into the voting booth "...every four years on the Tuesday after the first Monday in November" (Office Of Federal Register 6), the people of the United States of America expect their vote to count. Although, the president is elected in a more indirect way, by the electoral college, which does not represent the people, and is not fair. The president should be elected by popular vote from now on...]

This essay demonstrates strong argumentation, clear structure, and effective use of evidence. Note the clear thesis statement, logical organization, and proper citation of sources. The analysis is thorough and considers multiple aspects of the electoral college system.

POOR ESSAY EXAMPLE:

[When sitting in class do you ever feel like you can read someones emotion, well now you can. With the new program Facial Action Coding System you can make a 3-D model of someone and scan the face mucsles to tell you what emotions they are feeling...]

This essay shows weaknesses in organization, lacks proper evidence, and has multiple mechanical errors. Note the informal tone, lack of clear thesis, and absence of proper paragraph structure and transitions.

Please grade the following essay:

{essay_text}"""
)

ValueError: Single '{' encountered in format string

In [ ]:
#Fayez
from langchain.llms import LlamaCpp
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

essay_grader_prompt = PromptTemplate(
    input_variables=["essay_text"],
    template="""
You are an experienced essay grader. When given an essay, analyze it according to the following rubric, scoring each criterion from 0-10:

1. Thesis & Main Argument
- Clear, specific thesis statement
- Consistent focus throughout the essay
- Well-developed main arguments

2. Evidence & Support
- Relevant examples and evidence
- Proper citation of sources
- Effective integration of quotes/evidence

3. Organization & Structure
- Logical flow between paragraphs
- Strong topic sentences
- Effective transitions
- Clear introduction and conclusion

4. Critical Thinking
- Depth of analysis
- Consideration of counterarguments
- Original insights
- Logical reasoning

5. Writing Mechanics
- Grammar and spelling
- Sentence structure variety
- Academic vocabulary
- Proper formatting

For each essay, provide a structured and detailed evaluation as follows:

1. Summary: A brief (2-3 sentences) summary of the main arguments and key points of the essay.
2. Rubric Scores: Assign a score (0-10) for each criterion, along with a concise justification.
3. Overall Score: Calculate the total score out of 50, ensuring fairness and consistency.
4. Strengths: Identify three specific strengths of the essay.
5. Areas for Improvement: Provide three actionable suggestions, including concrete examples for implementation.
6. Final Feedback: A concluding paragraph with constructive feedback, focusing on the most impactful ways the writer can improve their next essay.

Maintain a constructive and supportive tone, highlighting both strengths and areas for improvement without discouraging the writer.

Please grade the following essay:

{essay_text}

"""
)

chain = LLMChain(llm=llm, prompt=essay_grader_prompt)
result = chain.run(essay_text=essay_text)

# essay_text=print(input("Enter your essay: "))

<ipython-input-6-96819a153b14>:59: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(essay_text=essay_text)


In [ ]:
essay_text="""Formative experience no. 1: moot courts
The Realities of Competing in Russia
Moot courts were the most formative experiences of my university years. To grasp the significance, it's important to understand what a moot court meant for a student in a post-Soviet university compared to one at a Western institution, or at least, how it was when I participated.

1. Moot courts were the only interesting thing to do
Moot courts weren't just one option among many; they were the only truly interesting thing to do. The alternatives included daily lectures and seminars (3 to 5 per day), student conferences (I participated in 11 and eventually lost enthusiasm), and internships (typically sought in your 4th or 5th year if you weren’t doing a moot court). Beyond that, the options were sparse.

2. Moot courts were a means of upward social mobility
Moot courts were also a potential ladder for upward social mobility. Participating could earn you an interview with a law firm that you wouldn’t get otherwise.

More importantly, it could expose you to new ways of thinking and introduce you to a reality you may not have known existed – opportunities to go abroad, pursue a master’s degree, or build an international career.

3. Moot courts were a scarce resource
At my university, moot courts were incredibly scarce. Only two were typically advertised – the Willem C. Vis International Commercial Arbitration Moot and the Philip C. Jessup International Law Moot Court Competition. Coaches prioritized winning over giving opportunities to more students, often selecting those who had participated before. This meant that out of 5 to 6 spots on the team, only 2 or 3 might be available to new participants. Considering the law faculty admitted about 500 people annually, the competition for these few spots was intense."""

In [ ]:
print(result)

### Summary
The essay discusses the author's formative experiences with moot courts during their university years in a post-Soviet context. It highlights three main points: the lack of engaging alternatives to moot courts, the potential for upward social mobility through participation, and the scarcity of opportunities available to students. The author emphasizes the unique significance of moot courts in shaping their academic and professional trajectory.

### Rubric Scores

1. **Thesis & Main Argument (7/10)**  
   The thesis is implied rather than explicitly stated, focusing on the importance of moot courts. While the main arguments are present, they could be more clearly articulated and connected to a central thesis.

2. **Evidence & Support (6/10)**  
   The essay provides relevant examples, such as the limited availability of moot courts and their role in social mobility. However, it lacks citations or references to support claims, which weakens the overall argument.

3. **Organiz

In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=1008d7905ba706693c02de9f4fd437f9fea2dcdec72809de68f520b920d981b5
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
#Agent
from langchain.agents import AgentType, initialize_agent
from langchain.tools import tool
from langchain.utilities import SerpAPIWrapper
from langchain.prompts import PromptTemplate

# Plagiarism detection tool
serpapi_api_key="31d166d1579b1e152efb9f7e71cd1a55151f819af90d9eb9e4cb1d700a6bbaec"
@tool
def plagiarism_check(text: str) -> str:
    """Checks if an essay is plagiarized by searching the internet for similar content."""

    # 🔹 Step 1: Search the web
    search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)
    web_results = search.run(text)

    # 🔹 Step 2: Analyze with GPT-4o-mini
    if not web_results:
        return "No plagiarism detected. No matching content found online."

    analysis_prompt = PromptTemplate(
    input_variables=["text", "web_results"],
    template=("""
        Analyze the following web search results and determine if the input text is plagiarized.\n\n
        Essay:\n{text}\n\n
        Web Results:\n{web_results}\n\n
        Should we flag this as plagiarism? If yes, explain why

        at the end give a score out of 100% is this essay is plagiarized
        """
    )
)

    formatted_prompt = analysis_prompt.format(text=text, web_results=web_results)
    result = llm.predict(formatted_prompt)

    return result

# Initialize agent
tools = [plagiarism_check]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [ ]:

essay_text = "Artificial Intelligence is transforming education by providing personalized learning experiences."
agent.run(f"Check if this essay is plagiarized: {essay_text}")




> Entering new AgentExecutor chain...
I need to check if the provided text is plagiarized by searching for similar content on the internet. 
Action: plagiarism_check
Action Input: "Artificial Intelligence is transforming education by providing personalized learning experiences."

<ipython-input-6-6eded8137a7e>:35: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm.predict(formatted_prompt)



Observation: Based on the analysis of the provided essay and the web search results, the input text states: "Artificial Intelligence is transforming education by providing personalized learning experiences." 

The web search results contain several phrases that are quite similar in meaning and structure to the input text. For example:

1. "AI is revolutionizing education by tailoring learning experiences to individual students' needs..."
2. "Artificial intelligence is revolutionizing the education sector, providing personalized learning experiences..."
3. "By leveraging technologies... AI can tailor personalized..."

These results indicate that the concept of AI transforming education through personalized learning is a common theme in existing literature. However, the specific phrasing of the input text is not an exact match to any of the web results. 

While the idea presented in the essay is widely discussed and similar to the content found in the search results, the sentence itself

'The essay does not contain direct plagiarism, as it does not copy any specific phrases or sentences from existing sources. However, it expresses a widely recognized idea about AI in education, resulting in a minimal risk of plagiarism (10%).'